# N-body problem

In [ ]:
import mylib.integration as integration
from mylib.model import three_body_model

import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import output_notebook
from bokeh.plotting import show, figure
from bokeh.palettes import Category20

import warnings
warnings.filterwarnings('ignore')

output_notebook(hide_banner=True)

In [ ]:
def plot_sol():

    m1 = 9.54786104043e-4
    m2 = 2.85583733151e-4

    qini = np.zeros(9)
    qini[0] =  0.
    qini[1] =  0.
    qini[2] =  0.
    qini[3] = -3.5023653
    qini[4] = -3.8169847
    qini[5] = -1.5507963
    qini[6] =  9.0755314
    qini[7] = -3.0458353
    qini[8] = -1.6483708

    pini = np.zeros(9)
    pini[0] =  0.
    pini[1] =  0.
    pini[2] =  0.
    pini[3] =  0.00565429*m1
    pini[4] = -0.00412490*m1
    pini[5] = -0.00190589*m1
    pini[6] =  0.00168318*m2
    pini[7] =  0.00483525*m2
    pini[8] =  0.00192462*m2

    yini = np.concatenate((qini, pini))

    tini = 0.
    tend = 12500.
    nt = 1501

    tbm = three_body_model()
    H_p = tbm.H_p
    H_q = tbm.H_q
    fcn = tbm.fcn

    fig_xy = figure(plot_height=500, plot_width=950, title="xy (Click on legend on hide corresponding graph)")

    # forward Euler integration
    sol_fe = integration.forward_euler(tini, tend, nt, yini, fcn)
    yfe = sol_fe.y
    fig_xy.x(yfe[3]-yfe[0], yfe[4]-yfe[1], color=Category20[20][0], legend="forward Euler")
    fig_xy.x(yfe[6]-yfe[0], yfe[7]-yfe[1], color=Category20[20][1], legend="forward Euler")

    # backward Euler integration
    sol_be = integration.backward_euler(tini, tend, nt, yini, fcn)
    ybe = sol_be.y
    fig_xy.x(ybe[3]-ybe[0], ybe[4]-ybe[1], color=Category20[20][8], legend="backward Euler")
    fig_xy.x(ybe[6]-ybe[0], ybe[7]-ybe[1], color=Category20[20][9], legend="backward Euler")

    # symplectic Euler 
    sol_se = integration.symplectic_euler(tini, tend, nt, yini, H_p, H_q)
    yse = sol_se.y
    fig_xy.x(yse[3]-yse[0], yse[4]-yse[1], color=Category20[20][2], legend="symplectic Euler")
    fig_xy.x(yse[6]-yse[0], yse[7]-yse[1], color=Category20[20][3], legend="symplectic Euler")

    # Stormer-Verlet
    sol_sv = integration.stormer_verlet(tini, tend, nt, yini, H_p, H_q)
    ysv = sol_sv.y
    fig_xy.x(ysv[3]-ysv[0], ysv[4]-ysv[1], color=Category20[20][4], legend="Stormer-Verlet")
    fig_xy.x(ysv[6]-ysv[0], ysv[7]-ysv[1], color=Category20[20][5], legend="Stormer-Verlet")

    # optimized order 8 step 15 integration
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    yo815 = sol_o815.y
    fig_xy.x(yo815[3]-yo815[0], yo815[4]-yo815[1], color=Category20[20][6], legend="optimized order 8 step 15")
    fig_xy.x(yo815[6]-yo815[0], yo815[7]-yo815[1], color=Category20[20][7], legend="optimized order 8 step 15")

    fig_xy.legend.background_fill_alpha = 0.75
    fig_xy.legend.click_policy="hide"
    show(fig_xy)
    
plot_sol()

In [ ]:
def show_invariant():

    m0 = 1.00000597682
    m1 = 9.54786104043e-4
    m2 = 2.85583733151e-4
    
    G = 2.95912208286e-4

    qini = np.zeros(9)
    qini[0] =  0.
    qini[1] =  0.
    qini[2] =  0.
    qini[3] = -3.5023653
    qini[4] = -3.8169847
    qini[5] = -1.5507963
    qini[6] =  9.0755314
    qini[7] = -3.0458353
    qini[8] = -1.6483708

    pini = np.zeros(9)
    pini[0] =  0.
    pini[1] =  0.
    pini[2] =  0.
    pini[3] =  0.00565429*m1
    pini[4] = -0.00412490*m1
    pini[5] = -0.00190589*m1
    pini[6] =  0.00168318*m2
    pini[7] =  0.00483525*m2
    pini[8] =  0.00192462*m2

    yini = np.concatenate((qini, pini))

    tini = 0.
    tend = 100000.
    nt = 5001
    dt = (tend-tini)/(nt-1)

    tbm = three_body_model()
    H_p = tbm.H_p
    H_q = tbm.H_q
    fcn = tbm.fcn
    
    fig_ham = figure(x_range=(tini, tend), plot_height=500, plot_width=950, title="Hamiltonian")
    t = np.linspace(tini, tend, nt)

    sol_se = integration.symplectic_euler(tini, tend, nt, yini, H_p, H_q)
    ham_se = tbm.hamiltonian(sol_se.y)
    plt_ham_se = fig_ham.x(sol_se.t, ham_se, color="red", legend="symplectic Euler")
    
    sol_sv = integration.stormer_verlet(tini, tend, nt, yini, H_p, H_q)
    ham_sv = tbm.hamiltonian(sol_sv.y)
    plt_ham_sv = fig_ham.line(sol_sv.t, ham_sv, color="blue", legend="Stormer-Verlet")
    
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    ham_o815 = tbm.hamiltonian(sol_o815.y)
    plt_ham_o815 = fig_ham.x(sol_o815.t, ham_o815, color="green", legend="optimized order 8 step 15")
    
    sol_dop5 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-6, atol=1.e-6)
    ham_dopri5 = tbm.hamiltonian(sol_dop5.y)
    fig_ham.x(sol_dop5.t, ham_dopri5, color="purple", legend="dopri5")

    fig_ham.legend.background_fill_alpha = 0.75
    fig_ham.legend.click_policy="hide"
    show(fig_ham)
        
show_invariant()